In [70]:
import pandas as pd
import json
import os # Operaciones relacionadas con el sistema de archivos
import datetime

### Metadatos

In [71]:
# Función para leer y filtrar los archivos JSON
def filtrar_archivos_json(directorio, categoria_busqueda):
    ''' Esta función, llamada filtrar_archivos_json, se encarga de buscar y filtrar 
        archivos JSON en un directorio específico en función de una categoría de búsqueda 
        proporcionada.   '''
    resultados = []
    for archivo in os.listdir(directorio):
            ruta_archivo = os.path.join(directorio, archivo)

            # Leer el contenido del archivo JSON
            with open(ruta_archivo) as f:
                archivo_json = f.readlines()

            # Cargar archivo JSON linea por linea
            for linea in archivo_json:
                data = json.loads(linea)
                # Verificar si la categoría buscada está presente en el archivo JSON
                categorias = data.get("category", [])
                if isinstance(categorias, list) and any(categoria_busqueda.lower() in categoria.lower() for categoria in categorias):
                    resultados.append(data)
    return resultados


In [72]:
# Filtrar los archivos JSON para la categoría "restaurant"
restaurant = filtrar_archivos_json('../PROYECTO_FINAL_EQUIPO/Google/metadata-sitios', "restaurant")
# Convertir los resultados a un pandas DataFrame
df_restaurant = pd.DataFrame(restaurant)

In [73]:
df_restaurant.shape

(121637, 15)

In [74]:
df_restaurant.head(2)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Fresh Off The Truck,"Fresh Off The Truck, Mueller Trailer Eatery, 4...",0x8644b5f995211beb:0x489b703481320404,"Food truck located at Mueller Trailer Eats, se...",30.296518,-97.707304,"[Caterer, Restaurant]",5.0,8,None,None,{'Service options': ['Delivery']},None,"[0x8644b5f3d971856f:0x3e04f8904bf2b2ee, 0x8644...",https://www.google.com/maps/place//data=!4m2!3...
1,Papa Joe's BBQ Co.,"Papa Joe's BBQ Co., 24415 NE 10th Ave, Ridgefi...",0x5495aef37bce66d5:0xfcf97974639cd5dd,None,45.798254,-122.660751,[Restaurant],4.6,14,None,"[[Thursday, 12–7PM], [Friday, 12–7PM], [Saturd...",{'From the business': ['Identifies as women-le...,Permanently closed,"[0x5495b2cea4d3a7f7:0xd788b88074f97be6, 0x5495...",https://www.google.com/maps/place//data=!4m2!3...


In [75]:
#1061 restaurantes duplicados
df_restaurant['gmap_id'].duplicated().sum()

1061

In [76]:
#Revisión de restaurantes duplicados
restaurantes_duplicados = df_restaurant[df_restaurant['gmap_id'].duplicated(keep=False)]
# Ordena los restaurantes duplicados por gmap_id
restaurantes_duplicados_ordenados = restaurantes_duplicados.sort_values(by='gmap_id')
restaurantes_duplicados_ordenados.head(2)


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
36445,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...
37506,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...


In [77]:
#Paso 1: Eliminar duplicados
df_restaurant = df_restaurant.drop_duplicates(subset='gmap_id') 

In [78]:
#Tenemos 120,575 restaurantes con 15 atributos, de los cuales es necesario cuales tienen
#realmente reseñas para los futuros procesos de ML
df_restaurant.shape

(120576, 15)

### Revisión de algunas columnas a eliminar

In [79]:
# relative_results hace referencia de recomendación a otros negocios
# pero dado que nos interesa hacer un sistema propio de recomendaciones, lo dejaremos de lado.
df_restaurant['relative_results'][1] 

['0x5495b2cea4d3a7f7:0xd788b88074f97be6',
 '0x5495a8ba9e39a85f:0xfbe5938763a14568',
 '0x54944d01d969e923:0xf8cc1e24f8aaeb50',
 '0x54944d27a2b501fd:0xde3ce099feaa6d78',
 '0x5495a9b18b24d4d7:0x93026337fd559ca3']

In [80]:
#Muchos de los datos que 'relative_results' arroja no están en nuestro DF
df_restaurant[df_restaurant['gmap_id']== '0x5495b2cea4d3a7f7:0xd788b88074f97be6']

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url


In [81]:
#Nos dice el estado en el que se encuentra el restaurante al momento que se 
# extrajeron los datos por lo que no es relevante para nuestro análisis
df_restaurant['state'][1]

'Permanently closed'

In [82]:
df_restaurant.drop(columns='state',inplace=True)
df_restaurant.drop(columns='relative_results',inplace=True)

### Reviews-estados

Se hace la carga de las Reviews para obtener las que pertenezcan a los restaurantes
y así mismo filtrar los restaurantes con Review. 

In [217]:
ruta_reviews_estados = "../PROYECTO_FINAL_EQUIPO/Google/reviews-estados/"
rutas_subfolders = [f.path for f in os.scandir(ruta_reviews_estados) if f.is_dir()]

In [218]:
def df_json(ruta_folder):
    '''Unión de archivos en un DataFrame '''
# Inicializa una lista para almacenar los objetos JSON de los archivos
    data = []
    
    # Itera a través de los archivos en la carpeta especificada
    for file_name in os.listdir(ruta_folder):
        # Construye la ruta completa de cada archivo
        file_path = os.path.join(ruta_folder, file_name)
        # Abre el archivo en modo lectura ('r')
        with open(file_path, 'r') as file:
            # Itera a través de las líneas del archivo
            for line in file:
                # Elimina espacios en blanco al principio y al final de la línea
                line = line.strip()
                # Intenta cargar la línea como un objeto JSON
                json_obj = json.loads(line)
                    # Agrega el objeto JSON a la lista de datos
                data.append(json_obj)

    # Crea un DataFrame de pandas a partir de la lista de datos
    df = pd.DataFrame(data)
    
    # Devuelve el DataFrame resultante
    return df

In [219]:
def df_json_general(subfolders):
    # Inicializa una lista para almacenar DataFrames individuales
    df_list = []
    
    # Itera a través de las subcarpetas especificadas
    for subfolder in subfolders:
        # Llama a la función read_json_files para procesar los archivos JSON en la subcarpeta
        df = df_json(subfolder)
        
        # Agrega el DataFrame resultante a la lista
        df_list.append(df)
    
    # Devuelve una lista de DataFrames, uno por cada subcarpeta
    return df_list

In [185]:

lista_df = df_json_general(rutas_subfolders)

In [220]:
df_reviews = pd.concat(lista_df, ignore_index=True)

In [221]:
df_reviews.shape

(8879925, 8)

In [223]:
df_reviews.sample(2)

,user_id,name,time,rating,text,pics,resp,gmap_id
7200075,103804428393146158238,R E,1499921265455,5,None,None,None,0x89b646f67680bb55:0xe0619a3533b1b66b
5307450,109958927208261118778,Teresa g Gr.com@ Gmail.com Grace,1573758158921,5,I Placed my EXPERIENCE On Facebook instead of ...,None,"{'time': 1575316884978, 'text': 'Thank you for...",0x80c8c7eabedeab3d:0xf8d5353b2ea83ac7


#### SELECCIÓN DE RESTAURANTES CON RESEÑAS

In [224]:
#Obtenemos los establecimientos que tienen Reviews
id_establecimientos_con_reviews = df_reviews["gmap_id"].unique()

In [225]:
#Se tienen 142,234 establecimientos únicos o que tienen una review
id_establecimientos_con_reviews.shape[0]

142234

In [226]:
df_restaurant.shape

(120576, 13)

In [227]:
#Filtrar los restaurantes que tienen Reviews.
df_restaurantes = df_restaurant[df_restaurant["gmap_id"].isin(id_establecimientos_con_reviews)].reset_index(drop=True)

In [228]:
df_restaurantes.shape

(3223, 13)

#### Filtrar reseñas de Restaurantes

In [229]:
filtro=df_restaurantes['gmap_id'].unique()

In [230]:
df_reviews_res = df_reviews[df_reviews["gmap_id"].isin(filtro)].copy()

In [231]:
df_reviews_res.head(1)

,user_id,name,time,rating,text,pics,resp,gmap_id
196,104959831058524862747,Samuel Benson,1630018420109,4,A pretty seamless experience from top to botto...,None,None,0x874d9bf9d1db7c85:0xf1c3706a2b497a3


In [232]:
def convertir_a_datetime(timestamp_ms):
    fecha_hora = datetime.datetime.fromtimestamp(timestamp_ms / 1000.0)  # Dividir por 1000 para convertir a segundos
    return fecha_hora

In [233]:
# Aplicar la función a la columna 'time' y guardar el resultado en la misma columna
df_reviews_res['time'] = df_reviews_res['time'].apply(convertir_a_datetime)

# Ordenar el DataFrame por la columna 'time' de menor a mayor
df_reviews_res = df_reviews_res.sort_values(by='time')

In [234]:
#Dejar la fecha en formato año/mes/día: 
df_reviews_res['time'] = pd.to_datetime(df_reviews_res['time']).dt.date

### Revisión de las columnas Reviews y Avg_Raiting en el dataframe Restaurantes con los datos de REVIEWS.

In [235]:
df_rev=pd.DataFrame()


In [246]:
#Se obtienen la cantidad de Reviews en nuestros restaurantes y el average.
# Primero, calcula las columnas "num_of_reviews" y "avg_rating" en "RestaurantesReview".
df_rev['gmap_id']=df_reviews_res['gmap_id'].copy()
df_rev['num_of_reviews'] = df_reviews_res.groupby('gmap_id')['gmap_id'].transform('count')
df_rev['avg_rating'] = df_reviews_res.groupby('gmap_id')['rating'].transform('mean')
# Luego, crea un nuevo DataFrame con las columnas que necesitas de "RestaurantesReview".
df_nuevo = df_rev[['gmap_id', 'num_of_reviews', 'avg_rating']].drop_duplicates()

In [247]:
df_nuevo.sample(2)

,gmap_id,num_of_reviews,avg_rating
1379096,0x883a0a071c2a4f15:0x5cb6ef0eb2ef5c95,26.0,4.384615
2187375,0x880560545f107547:0x86b19d9367d79b6c,754.0,3.591512


In [256]:
df_reviews_res.sample(2)

,user_id,name,time,rating,text,pics,resp,gmap_id
1372864,105129062216865645402,Jack Storts,2019-03-01,5,Good deals great pizza,None,"{'time': 1552487888698, 'text': 'Thank you, Ja...",0x88389151a59be061:0x6d92b1ec86559f1c
5327236,105818227795390093857,Kenya Johnson,2017-11-21,1,Ummm! Just no.,None,None,0x80c8c22463636d17:0x7822da1f4d521f6f


Se puede ver que hay diferencia en el numero de "reviews" por consecuencia también en "avg_rating" y no se sabe la fecha en que fueron calculados los de "df_restaurantes", por lo que no sería correcto simplemente sumarlos y obtener su el promedio en avg_rating.
Por lo que se eliminaran. 

In [259]:
#RES avg_rating "4.5" & num_of_reviews "101" 0x88065334fbf8d89d:0xa8803652886f0bef	
#REV avg_rating "4.6" & num_of_reviews "76.0" 0x88065334fbf8d89d:0xa8803652886f0bef	
df_restaurantes.sample(1)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,url
2929,Distill - A Local Bar,"Distill - A Local Bar, 5750 S Decatur Blvd, La...",0x80c8c7c3de7c9ead:0x892a30757ade73a2,None,36.085341,-115.207441,"[Bar & grill, Bar, Restaurant]",4.5,101,None,"[[Saturday, Open 24 hours], [Sunday, Open 24 h...","{'Service options': ['Curbside pickup', 'Deliv...",https://www.google.com/maps/place//data=!4m2!3...


In [260]:
df_nuevo[df_nuevo['gmap_id'] == '0x80c8c7c3de7c9ead:0x892a30757ade73a2']

,gmap_id,num_of_reviews,avg_rating
5906744,0x80c8c7c3de7c9ead:0x892a30757ade73a2,76.0,4.684211


In [262]:
df_restaurantes.drop(columns='num_of_reviews',inplace=True) #Depende del periodo de tiempo y del DS Reviews
df_restaurantes.drop(columns='avg_rating',inplace=True) #Depende del periodo de tiempo y del DS Reviews

In [263]:
df_restaurantes.head(2)

,name,address,gmap_id,description,latitude,longitude,category,price,hours,MISC,url
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",0x88008ff5b3c52fd3:0xdbe482cf9fb221db,None,44.306954,-89.845783,[Restaurant],None,"[[Thursday, 12–11PM], [Friday, 12PM–12AM], [Sa...","{'Service options': ['Takeout', 'Dine-in', 'De...",https://www.google.com/maps/place//data=!4m2!3...
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...",0x80c8db636d525e3b:0x6a51cce4b2dcc350,None,36.164633,-115.060780,"[Mexican restaurant, Crêperie, Ice cream shop,...",None,"[[Thursday, 9AM–9PM], [Friday, 9AM–9PM], [Satu...","{'Service options': ['Delivery'], 'Amenities':...",https://www.google.com/maps/place//data=!4m2!3...


In [264]:
df_restaurantes.to_csv('Restaurantes_Google.csv',index=False)

In [265]:
df_reviews_res.to_csv('Restaurantes_Google_Reviews.csv',index=False)

Revision

In [266]:
df_reviews_res.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
2893620,113884074810430895487,JC Clark,2017-12-11,5,If you haven't tried their Mac & Cheese yet yo...,None,None,0x89e24f68777466cb:0x305a1ce4b468b584
2906304,117358624895753554376,Roland Martel,2020-10-25,4,Meal was ok have had better there. Onion soup ...,None,None,0x4cb344da481beb03:0x30c538e43dbf1a23
2076284,107504813527152601771,Abbey Brake,2018-09-03,4,None,None,None,0x8804251178df2741:0xdd7ce43feba08427


In [267]:
df_restaurantes[df_restaurantes['gmap_id']=='0x89e24f68777466cb:0x305a1ce4b468b584']

,name,address,gmap_id,description,latitude,longitude,category,price,hours,MISC,url
532,Mr. Mac’s Macaroni and Cheese,"Mr. Mac’s Macaroni and Cheese, 497 Hooksett Rd...",0x89e24f68777466cb:0x305a1ce4b468b584,"Ample varieties of mac ’n’ cheese including ""t...",43.01404,-71.451466,"[American restaurant, Catering food and drink ...",$$,"[[Tuesday, 11AM–9PM], [Wednesday, 11AM–9PM], [...","{'Service options': ['Curbside pickup', 'Deliv...",https://www.google.com/maps/place//data=!4m2!3...


In [268]:
df_reviews_res[df_reviews_res['gmap_id']=='0x89e24f68777466cb:0x305a1ce4b468b584'].shape

(1081, 8)

In [269]:
df_reviews_res.sample(2)

,user_id,name,time,rating,text,pics,resp,gmap_id
7049299,103953831837343316659,Arash E,2018-05-13,5,We came here at the hotel's recommendation. T...,None,None,0x89b5afb3d01303a1:0xe6dd68de1217d0c4
1428460,103427351158735334681,Mohammad Ayrout,2018-06-02,5,"The best authentic Arabic food in Cleveland, I...",None,None,0x8830ee047a73ffff:0x30563f42fe1d958


''' '''


In [270]:
df_reviews_res.shape

(152797, 8)